ECGR 5105 - HW6
Joshua Ayers
Github:

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import torch
import torch.nn as nn
import torch.optim as optim

Load the housing dataset and preprocess

In [2]:
# Load the dataset
df = pd.read_csv('Housing.csv')
df.replace({'yes': 1, 'no': 0}, inplace=True)

# Separate features and target
X = df.drop(columns=['price'])
y = df['price']

# List of categorical columns
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'furnishingstatus']
numerical_cols = X.columns.difference(categorical_cols)

# Preprocessing pipeline for categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Apply preprocessing to the data
X_processed = preprocessor.fit_transform(X)

# Convert to tensors
X_processed_tensor = torch.tensor(X_processed.astype(np.float32))
y_tensor = torch.tensor(y.values.astype(np.float32))

# Split the data into 80% training and 20% validation
X_train_tensor, X_val_tensor, y_train_tensor, y_val_tensor = train_test_split(X_processed_tensor, y_tensor, test_size=0.2, random_state=42)

# Convert y tensors to the correct shape
y_train_tensor = y_train_tensor.view(-1, 1)
y_val_tensor = y_val_tensor.view(-1, 1)

Define the Neural network For problem 1 part 1

In [3]:
# Define the neural network model with one hidden layer of size 32
class NeuralNetwork1(nn.Module):
    def __init__(self,hidden_layer_size = 32):
        super(NeuralNetwork1, self).__init__()
        self.fc1 = nn.Linear(X_train_tensor.shape[1], hidden_layer_size)
        self.fc2 = nn.Linear(hidden_layer_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

declare the network and define the 

In [4]:
# Initialize the model, loss function, and optimizer
model1 = NeuralNetwork1()

criterion = nn.MSELoss()
optimizer = optim.Adam(model1.parameters(), lr=0.0001)


# Convert data to torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)


NameError: name 'X_train' is not defined

In [ ]:
# Training loop
num_epochs = 1000


for epoch in range(num_epochs):
    model1.train()
    optimizer.zero_grad()
    outputs = model1(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
# Evaluate the model on the validation set
model1.eval()
with torch.no_grad():
    val_outputs = model1(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)
    print(f'Validation Loss: {val_loss.item():.4f}')


Epoch [100/1000], Loss: 25234788843520.0000
Epoch [200/1000], Loss: 25234788843520.0000
Epoch [300/1000], Loss: 25234784649216.0000
Epoch [400/1000], Loss: 25234780454912.0000
Epoch [500/1000], Loss: 25234774163456.0000
Epoch [600/1000], Loss: 25234769969152.0000
Epoch [700/1000], Loss: 25234765774848.0000
Epoch [800/1000], Loss: 25234759483392.0000
Epoch [900/1000], Loss: 25234746900480.0000
Epoch [1000/1000], Loss: 25234738511872.0000
Validation Loss: 30129933778944.0000


In [ ]:
# Evaluate the model on the validation set
model1.eval()
with torch.no_grad():
    val_outputs = model1(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)
    print(f'Validation Loss: {val_loss.item():.4f}')


Validation Loss: 30129933778944.0000


In [ ]:
# Train a linear regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [ ]:

# Predict and evaluate on validation set
y_val_pred_lr = lr_model.predict(X_val)
lr_val_loss = mean_squared_error(y_val, y_val_pred_lr)
print(f'Linear Regression Validation Loss: {lr_val_loss:.4f}')

# Compare model complexity (number of trainable parameters)
nn_params = sum(p.numel() for p in model1.parameters() if p.requires_grad)
lr_params = X_train.shape[1] + 1  # Weights + Bias
print(f'Neural Network Parameters: {nn_params}')
print(f'Linear Regression Parameters: {lr_params}')


Linear Regression Validation Loss: 1739553620722.7549
Neural Network Parameters: 769
Linear Regression Parameters: 23


In [ ]:
# Define the neural network
class MyModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Example model instantiation with sizes 16, 32, and 64
input_size = X_processed.shape[1]
hidden_size1 = 32
hidden_size2 = 64
output_size = 1  # For regression

model2 = MyModel(input_size, hidden_size1, hidden_size2, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

num_epochs = 1000
for epoch in range(num_epochs):
    model2.train()
    optimizer.zero_grad()
    outputs = model2(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the validation set
model2.eval()
with torch.no_grad():
    val_outputs = model2(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)
    print(f'Validation Loss: {val_loss.item():.4f}')

Epoch [100/1000], Loss: 25234428133376.0000
Epoch [200/1000], Loss: 25230036697088.0000
Epoch [300/1000], Loss: 25215516016640.0000
Epoch [400/1000], Loss: 25185310736384.0000
Epoch [500/1000], Loss: 25134540783616.0000
Epoch [600/1000], Loss: 25058934259712.0000
Epoch [700/1000], Loss: 24954823245824.0000
Epoch [800/1000], Loss: 24819072499712.0000
Epoch [900/1000], Loss: 24649073164288.0000
Epoch [1000/1000], Loss: 24442711310336.0000
Validation Loss: 29224341602304.0000


In [ ]:
# Initialize the model, loss function, and optimizer



In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    model2.train()
    optimizer.zero_grad()
    outputs = model2(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
# Evaluate the model on the validation set
model1.eval()
with torch.no_grad():
    val_outputs = model2(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor)
    print(f'Validation Loss: {val_loss.item():.4f}')

Epoch [100/1000], Loss: 24198344867840.0000
Epoch [200/1000], Loss: 23914812014592.0000
Epoch [300/1000], Loss: 23591378747392.0000
Epoch [400/1000], Loss: 23227732590592.0000
Epoch [500/1000], Loss: 22823982596096.0000
Epoch [600/1000], Loss: 22380634177536.0000
Epoch [700/1000], Loss: 21898551361536.0000
Epoch [800/1000], Loss: 21378971467776.0000
Epoch [900/1000], Loss: 20823465263104.0000
Epoch [1000/1000], Loss: 20233922281472.0000
Validation Loss: 24409532268544.0000


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Load the dataset
df = pd.read_csv('Housing.csv')
df.replace({'yes': 1, 'no': 0}, inplace=True)

# Separate features and target
X = df.drop(columns=['price'])
y = df['price']

# List of categorical columns
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'furnishingstatus']
numerical_cols = X.columns.difference(categorical_cols)

# Preprocessing pipeline for categorical and numerical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# Apply preprocessing to the data
X_processed = preprocessor.fit_transform(X)

# Convert to tensors
X_processed_tensor = torch.tensor(X_processed.astype(np.float32))
y_tensor = torch.tensor(y.values.astype(np.float32)).view(-1, 1)

# Split the data into 80% training and 20% validation
X_train_tensor, X_val_tensor, y_train_tensor, y_val_tensor = train_test_split(X_processed_tensor, y_tensor, test_size=0.2, random_state=42)

# Define the neural network with one hidden layer of size 32
class SingleLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SingleLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the neural network with three hidden layers of sizes 32, 64, and 16
class MultiLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, output_size):
        super(MultiLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize models
input_size = X_processed.shape[1]
output_size = 1

single_layer_model = SingleLayerNN(input_size, 32, output_size)
multi_layer_model = MultiLayerNN(input_size, 32, 64, 16, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
single_layer_optimizer = optim.Adam(single_layer_model.parameters(), lr=0.001)
multi_layer_optimizer = optim.Adam(multi_layer_model.parameters(), lr=0.001)

# Training function
def train_model(model, optimizer, X_train, y_train, num_epochs=1000):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Train single layer model
print("Training single layer model...")
train_model(single_layer_model, single_layer_optimizer, X_train_tensor, y_train_tensor)

# Evaluate single layer model
single_layer_model.eval()
with torch.no_grad():
    single_val_outputs = single_layer_model(X_val_tensor)
    single_val_loss = criterion(single_val_outputs, y_val_tensor)
    print(f'Single Layer Model Validation Loss: {single_val_loss.item():.4f}')

# Train multi-layer model
print("Training multi-layer model...")
train_model(multi_layer_model, multi_layer_optimizer, X_train_tensor, y_train_tensor)

# Evaluate multi-layer model
multi_layer_model.eval()
with torch.no_grad():
    multi_val_outputs = multi_layer_model(X_val_tensor)
    multi_val_loss = criterion(multi_val_outputs, y_val_tensor)
    print(f'Multi Layer Model Validation Loss: {multi_val_loss.item():.4f}')

# Compare model complexity
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

single_layer_params = count_parameters(single_layer_model)
multi_layer_params = count_parameters(multi_layer_model)

print(f'Single Layer Model Parameters: {single_layer_params}')
print(f'Multi Layer Model Parameters: {multi_layer_params}')


Training single layer model...
Epoch [100/1000], Loss: 25234763677696.0000
Epoch [200/1000], Loss: 25234658820096.0000
Epoch [300/1000], Loss: 25234463784960.0000
Epoch [400/1000], Loss: 25234180669440.0000
Epoch [500/1000], Loss: 25233809473536.0000
Epoch [600/1000], Loss: 25233339711488.0000
Epoch [700/1000], Loss: 25232773480448.0000
Epoch [800/1000], Loss: 25232123363328.0000
Epoch [900/1000], Loss: 25231395651584.0000
Epoch [1000/1000], Loss: 25230592442368.0000
Single Layer Model Validation Loss: 30125213089792.0000
Training multi-layer model...
Epoch [100/1000], Loss: 25233482317824.0000
Epoch [200/1000], Loss: 25194280255488.0000
Epoch [300/1000], Loss: 24962693857280.0000
Epoch [400/1000], Loss: 24257945927680.0000
Epoch [500/1000], Loss: 22738798379008.0000
Epoch [600/1000], Loss: 20128127254528.0000
Epoch [700/1000], Loss: 16393414115328.0000
Epoch [800/1000], Loss: 11923227672576.0000
Epoch [900/1000], Loss: 7546842644480.0000
Epoch [1000/1000], Loss: 4204868927488.0000
Mul